# Installing required libraries

In [ ]:
! pip install tensorflow
! pip install transformers

# Importing required libraries

In [ ]:
import pandas as pd
import tensorflow as tf
import torch
import numpy as np
import requests
import cv2
import seaborn as sea
import matplotlib.pyplot as plt

from transformers import GPT2Tokenizer, GPT2Model
# from tensorflow.keras import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Loading Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/multimodal_test_public.tsv', delimiter='\t')
df = df.head(10000)

# Extracting labels from the dataset


In [ ]:
labels = df['2_way_label'].values

# Initializing the tokenizer and model for text processing


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2Model.from_pretrained('gpt2')

# Splitting the dataset into training and testing sets


In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(df[['clean_title','image_url','2_way_label']], labels, test_size=0.2, random_state=42)

NameError: ignored

# Function to get text embeddings using GPT2

In [ ]:
def get_text_embedding(text):
    tokenized_text = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        model_output = model(tokenized_text)
    embedding = model_output.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

# Function to get image embeddings from a URL

In [ ]:
def get_image_embedding(url, target_size=(224, 224)):
    try:
        response = requests.get(url)
        response.raise_for_status()
        image = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)

        if image is None:
            raise Exception("Failed to decode image from URL.")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, target_size)
        image_array = image.astype(np.float32) / 255.0
        return image_array

    except Exception as e:
        print(f"Error processing image from URL {url}: {str(e)}")
        return None

# Function to get text and image embeddings and labels from dataframe

In [ ]:
def get_text_embeddings_image_embeddings_labels(dataframe):
    text_embeddings, image_embeddings, labels = [], [], []
    for clean_title, image_url, label in zip(dataframe['clean_title'], dataframe['image_url'], dataframe['2_way_label']):
        text_embedding = get_text_embedding(clean_title)
        image_embedding = get_image_embedding(image_url, (224, 224))

        if text_embedding is not None and image_embedding is not None:
            text_embeddings.append(text_embedding)
            image_embeddings.append(image_embedding)
            labels.append(label)

    return np.array(text_embeddings), np.array(image_embeddings), labels

# Getting embeddings and labels for training and testing sets

In [ ]:
train_text_embeddings, train_image_embeddings, train_labels = get_text_embeddings_image_embeddings_labels(train_data)
test_text_embeddings, test_image_embeddings, test_labels = get_text_embeddings_image_embeddings_labels(test_data)

NameError: ignored

# Using Keras to define a neural network model to process text and image embeddings


In [ ]:
text_input = layers.Input(shape=(train_text_embeddings.shape[1],))
text_layer = layers.Dense(128, activation='swish')(text_input)

image_input = layers.Input(shape=(224, 224, 3))
image_layer = layers.Conv2D(64, (3, 3), activation='swish')(image_input)
image_layer = layers.MaxPooling2D((2, 2))(image_layer)
image_layer = layers.Flatten()(image_layer)

merged = layers.concatenate([text_layer, image_layer])
output = layers.Dense(1, activation='sigmoid')(merged)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

model = tf.keras.Model(inputs=[text_input,  image_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

NameError: ignored

# Training the model

In [ ]:
model.fit([train_text_embeddings, train_image_embeddings], train_labels, epochs=5, batch_size=32, validation_split=0.2)

# Evaluating the model


In [ ]:
accuracy = model.evaluate([test_text_embeddings, test_image_embeddings], test_labels)
print(f'Test Accuracy: {accuracy[1]*100:.2f}%')

# Predicting and generating a confusion matrix

In [ ]:
predictions = model.predict([test_text_embeddings, test_image_embeddings])
predicted_labels = (predictions > 0.5).astype(int)
confusion_matrix = confusion_matrix(test_labels, predicted_labels)
plt.figure(figsize=(10, 10))
sea.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Greens', xticklabels=[1,5],yticklabels=[1,5])
plt.title('Confusion Matrix - GPT')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

# Generating Classification Report

In [ ]:
report_gpt = classification_report(test_labels, predicted_labels)
print (f'\nClassification Report - GPT\n {report_gpt}')